# Purpose
- Purpose of this notebook is to;
    - read raw data
    - validate if time series' are continuous (fill missing if not continuous)
    - save validated time series'

In [1]:
# change current working directory to the root of the project
import os
os.chdir(os.path.dirname(os.getcwd()))
import pandas as pd
import numpy as np

from IPython.display import display

# Import Raw Data
**Data Source**: Kaggle, Store Sales - Time Series Forecasting
- **df_train**, comprising time series of features **store_nbr**, **family**, and **onpromotion** as well as the target **sales**.
    - **store_nbr** identifies the store at which the products are sold.
    - **family** identifies the type of product sold.
    - **sales** gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
    - **onpromotion** gives the total number of items in a product family that were being promoted at a store at a given date.
- **df_test**, having the same features as the training data. You will predict the target sales for the dates in this file.
    - The dates in the test data are for the 15 days after the last date in the training data.
- **df_stores**, Store metadata, including **city**, **state**, **type**, and **cluster**.
    - **cluster** is a grouping of similar stores.
- **df_oil**, Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)
- **df_holidays_events**, Holidays and Events, with metadata
    - NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
    - Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

**Additional Notes**
Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

In [2]:
# import csv data into pandas dataframe
df_train = pd.read_csv('data/raw/train.csv')
df_test = pd.read_csv('data/raw/test.csv')

df_holidays_events = pd.read_csv('data/raw/holidays_events.csv')
df_oil = pd.read_csv('data/raw/oil.csv')
df_stores = pd.read_csv('data/raw/stores.csv')
df_transactions = pd.read_csv('data/raw/transactions.csv')

In [3]:
print(f'{df_train.shape=}')
display(df_train.head())
print(f'{df_test.shape=}')
display(df_test.head())
print(f'{df_holidays_events.shape=}')
display(df_holidays_events.head())
print(f'{df_oil.shape=}')
display(df_oil.head())
print(f'{df_stores.shape=}')
display(df_stores.head())
print(f'{df_transactions.shape=}')
display(df_transactions.head())

df_train.shape=(3000888, 6)


,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


df_test.shape=(28512, 5)


,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


df_holidays_events.shape=(350, 6)


...

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


df_stores.shape=(54, 5)


,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


df_transactions.shape=(83488, 3)


,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


# Data Validation

- check if each 'family' in the train set & test set have the same number of rows

In [4]:
assert len(df_train[['family', 'store_nbr']].value_counts().unique()) == 1, 'each family does not have the same number of rows'
assert len(df_test[['family', 'store_nbr']].value_counts().unique()) == 1, 'each family does not have the same number of rows'

- check if each 'family' & 'store_nbr' in the train set & test set have the same number of rows

In [5]:
assert len(df_train.groupby(['family', 'store_nbr']).size().reset_index().rename(columns={0: 'count'})['count'].value_counts().unique()) == 1, 'each family does not have the same number of rows'
assert len(df_test.groupby(['family', 'store_nbr']).size().reset_index().rename(columns={0: 'count'})['count'].value_counts().unique()) == 1, 'each family does not have the same number of rows'

- check if time series is continuous

In [6]:
# df_train
df_train['date'] = pd.to_datetime(df_train['date'])
df_train.groupby(['family', 'store_nbr'])['date'].diff().dt.days.value_counts()
# looks like train set is not continuous

1.0    2991978
2.0       7128
Name: date, dtype: int64

In [7]:
# df_test
df_test['date'] = pd.to_datetime(df_test['date'])
df_test.groupby(['family', 'store_nbr'])['date'].diff().dt.days.value_counts()
# test set does not have any missing dates, and is therefore continuous

1.0    26730
Name: date, dtype: int64

In [8]:
# df_oil
df_oil['date'] = pd.to_datetime(df_oil['date'])
df_oil['date'].diff().dt.days.value_counts()
# oil data is not continuous

1.0    974
3.0    243
Name: date, dtype: int64

In [9]:
# check if there are any missing values in the sales column
df_train['sales'].isna().sum()

0

- looks like there are some missing dates in the train set & oil dataset
- check if the missing dates in the train set & oil dataset are due to holidays

In [11]:
df_train['date'] = pd.to_datetime(df_train['date'])

df_train_new = (df_train.set_index('date')
                .groupby(['family','store_nbr'])[['sales', 'onpromotion']]
                .apply(lambda x: x.asfreq('d', fill_value=np.nan))
                .reset_index())

df_train_new.groupby(['family', 'store_nbr'])['date'].diff().dt.days.value_counts()

1.0    3006234
Name: date, dtype: int64

In [12]:
df_oil['date'] = pd.to_datetime(df_oil['date'])

df_oil_new = df_oil.set_index('date').asfreq('d', fill_value=np.nan).reset_index()
df_oil_new

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-05,NaN
...,...,...
1699,2017-08-27,NaN
1700,2017-08-28,46.40
1701,2017-08-29,46.46
1702,2017-08-30,45.96


In [13]:
# new dates have nan sales
df_train_new['sales'].isna().sum()

7128

In [14]:
df_oil_new['dcoilwtico'].isna().sum()

529

In [15]:
# merge train set with holidays
df_holidays_events['date'] = pd.to_datetime(df_holidays_events['date'])
df_train_new = df_train_new.merge(df_holidays_events, how='left', on='date')
df_train_new


,family,store_nbr,date,sales,onpromotion,type,locale,locale_name,description,transferred
0,AUTOMOTIVE,1,2013-01-01,0.0,0.0,Holiday,National,Ecuador,Primer dia del ano,False
1,AUTOMOTIVE,1,2013-01-02,2.0,0.0,NaN,NaN,NaN,NaN,NaN
2,AUTOMOTIVE,1,2013-01-03,3.0,0.0,NaN,NaN,NaN,NaN,NaN
3,AUTOMOTIVE,1,2013-01-04,3.0,0.0,NaN,NaN,NaN,NaN,NaN
4,AUTOMOTIVE,1,2013-01-05,5.0,0.0,Work Day,National,Ecuador,Recupero puente Navidad,False
...,...,...,...,...,...,...,...,...,...,...
3061471,SEAFOOD,54,2017-08-11,0.0,0.0,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False
3061472,SEAFOOD,54,2017-08-12,1.0,1.0,NaN,NaN,NaN,NaN,NaN
3061473,SEAFOOD,54,2017-08-13,2.0,0.0,NaN,NaN,NaN,NaN,NaN
3061474,SEAFOOD,54,2017-08-14,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [16]:
# merge train set with holidays
df_holidays_events['date'] = pd.to_datetime(df_holidays_events['date'])
df_oil_new = df_oil_new.merge(df_holidays_events, how='left', on='date')
df_oil_new


,date,dcoilwtico,type,locale,locale_name,description,transferred
0,2013-01-01,NaN,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-02,93.14,NaN,NaN,NaN,NaN,NaN
2,2013-01-03,92.97,NaN,NaN,NaN,NaN,NaN
3,2013-01-04,93.12,NaN,NaN,NaN,NaN,NaN
4,2013-01-05,NaN,Work Day,National,Ecuador,Recupero puente Navidad,False
...,...,...,...,...,...,...,...
1729,2017-08-27,NaN,NaN,NaN,NaN,NaN,NaN
1730,2017-08-28,46.40,NaN,NaN,NaN,NaN,NaN
1731,2017-08-29,46.46,NaN,NaN,NaN,NaN,NaN
1732,2017-08-30,45.96,NaN,NaN,NaN,NaN,NaN


In [17]:
# check if nan sales values correspond to holidays. If it corresponds, this could mean sales are 0 because the store is closed
df_train_new[df_train_new['sales'].isna()]['type'].value_counts(dropna=False)
# looks like this is the case

Holiday    7128
Name: type, dtype: int64

In [18]:
df_oil_new[df_oil_new['dcoilwtico'].isna()]['type'].value_counts(dropna=False)
# looks like nans in oil data do not correspond to specific holidays
# this means that the oil data is missing for some dates at random
# we can fill these nans with the previous oil value


NaN           439
Holiday        62
Event          22
Additional     14
Work Day        5
Transfer        2
Name: type, dtype: int64

In [19]:
# fill missing dates with sales = 0, since the store is closed due to a holiday
df_train['date'] = pd.to_datetime(df_train['date'])

df_train_filled = (df_train.set_index('date')
                .groupby(['family','store_nbr'])[['sales', 'onpromotion']]
                .apply(lambda x: x.asfreq('d', fill_value=0))
                .reset_index())

df_train_filled

,family,store_nbr,date,sales,onpromotion
0,AUTOMOTIVE,1,2013-01-01,0.0,0
1,AUTOMOTIVE,1,2013-01-02,2.0,0
2,AUTOMOTIVE,1,2013-01-03,3.0,0
3,AUTOMOTIVE,1,2013-01-04,3.0,0
4,AUTOMOTIVE,1,2013-01-05,5.0,0
...,...,...,...,...,...
3008011,SEAFOOD,54,2017-08-11,0.0,0
3008012,SEAFOOD,54,2017-08-12,1.0,1
3008013,SEAFOOD,54,2017-08-13,2.0,0
3008014,SEAFOOD,54,2017-08-14,0.0,0


In [20]:
df_oil['date'] = pd.to_datetime(df_oil['date'])

df_oil_filled = df_oil.set_index('date').asfreq('d').ffill().reset_index()

# fill first missing value with backfill (since it's just 1 value and it is the first value in the dataset, backfill is okay)
df_oil_filled['dcoilwtico'].fillna(method='bfill', inplace=True)
df_oil_filled


,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-05,93.12
...,...,...
1699,2017-08-27,47.65
1700,2017-08-28,46.40
1701,2017-08-29,46.46
1702,2017-08-30,45.96


# Final Checks

In [21]:
# check missing values
display(f'{df_train_filled.isna().sum()=}')
display(f'{df_test.isna().sum()=}')
display(f'{df_holidays_events.isna().sum()=}')
display(f'{df_oil_filled.isna().sum()=}')
display(f'{df_stores.isna().sum()=}')
display(f'{df_transactions.isna().sum()=}')

# no missing values in all datasets

'df_train_filled.isna().sum()=family         0\nstore_nbr      0\ndate           0\nsales          0\nonpromotion    0\ndtype: int64'

'df_test.isna().sum()=id             0\ndate           0\nstore_nbr      0\nfamily         0\nonpromotion    0\ndtype: int64'

'df_holidays_events.isna().sum()=date           0\ntype           0\nlocale         0\nlocale_name    0\ndescription    0\ntransferred    0\ndtype: int64'

'df_oil_filled.isna().sum()=date          0\ndcoilwtico    0\ndtype: int64'

'df_stores.isna().sum()=store_nbr    0\ncity         0\nstate        0\ntype         0\ncluster      0\ndtype: int64'

'df_transactions.isna().sum()=date            0\nstore_nbr       0\ntransactions    0\ndtype: int64'

In [22]:
df_train_filled['date'] = df_train_filled['date'].astype(str)
df_test['date'] = df_test['date'].astype(str)
df_oil_filled['date'] = df_oil_filled['date'].astype(str)
df_holidays_events['date'] = df_holidays_events['date'].astype(str)
df_transactions['date'] = df_transactions['date'].astype(str)

# max and min dates
print(f'{df_train_filled["date"].min()=} & {df_train_filled["date"].max()=}\n')
print(f'{df_test["date"].min()=} & {df_test["date"].max()=}\n')
print(f'{df_oil_filled["date"].min()=} & {df_oil_filled["date"].max()=}\n')
print(f'{df_holidays_events["date"].min()=} & {df_holidays_events["date"].max()=}\n')
print(f'{df_transactions["date"].min()=} & {df_transactions["date"].max()=}')


df_train_filled["date"].min()='2013-01-01' & df_train_filled["date"].max()='2017-08-15'

df_test["date"].min()='2017-08-16' & df_test["date"].max()='2017-08-31'

df_oil_filled["date"].min()='2013-01-01' & df_oil_filled["date"].max()='2017-08-31'

df_holidays_events["date"].min()='2012-03-02' & df_holidays_events["date"].max()='2017-12-26'

df_transactions["date"].min()='2013-01-01' & df_transactions["date"].max()='2017-08-15'


In [23]:
# check if all datasets have data for all stores
print(f'{df_train_filled["store_nbr"].nunique()=} & {df_test["store_nbr"].nunique()=} & {df_stores["store_nbr"].nunique()=} & {df_transactions["store_nbr"].nunique()=}')

df_train_filled["store_nbr"].nunique()=54 & df_test["store_nbr"].nunique()=54 & df_stores["store_nbr"].nunique()=54 & df_transactions["store_nbr"].nunique()=54


# Summary
- 1. train set is not continuous. These missing dates aligned with holidays, so we filled these missing dates with 0 sales
- 2. test set is continuous
- 3. oil data is not continuous. These missing dates did not align with holidays. So we filled these missing dates with the previous oil value. The first missing value in the dataset was backfilled.
- 4. no missing values in any dataset, after filling missing dates
- <br>
- 5. train set min date is 2013-01-01 and max date is 2017-08-15
- 6. test set min date is 2017-08-16 and max date is 2017-08-31
- 7. oil data min date is 2013-01-01 and max date is 2017-08-31
- 8. holidays data min date is 2012-03-02 and max date is 2017-12-26
- 9. transactions data min date is 2013-01-01 and max date is 2017-08-15


# Save preprocessed datasets

In [24]:
df_train_filled.to_csv('data/preprocessed/train_preprocessed.csv', index=False)
df_test.to_csv('data/preprocessed/test_preprocessed.csv', index=False)
df_oil_filled.to_csv('data/preprocessed/oil_preprocessed.csv', index=False)
df_holidays_events.to_csv('data/preprocessed/holidays_preprocessed.csv', index=False)
df_transactions.to_csv('data/preprocessed/transactions_preprocessed.csv', index=False)
df_stores.to_csv('data/preprocessed/stores_preprocessed.csv', index=False)